In [1]:
import argparse
import logging
import os, sys
from copy import copy
from datetime import datetime
from pathlib import Path
import time
from functools import partial
import gzip
from collections import defaultdict
from itertools import product
from functools import wraps

import dask.bag as db
import geopandas as gpd
import pandas as pd
import polars as pl
from tqdm import tqdm
from sqlalchemy.orm import Session
import networkx as nx
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import LineCollection
from matplotlib import colors as mcolors, cm, gridspec, ticker, patches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import string
alphabet = string.ascii_lowercase

from pathlib import Path

%reload_ext autoreload
%autoreload 2
%matplotlib inline

sys.path.append("..")
sys.path.append("../..")


from database import get_engine
from utils.config_utils import load_config
from utils import plot_utils
from utils import graph_utils

# Data setup


In [2]:
plot_conf_file = Path(".").resolve().parent / "config/plots/swiss-data/plot_data.yml"
plot_conf = load_config(plot_conf_file)

dbconf_file = Path(".").resolve().parent / "config/database/database.yaml"
dbconf = load_config(dbconf_file)
ENGINE = get_engine(dbconf, pool_size=5, max_overflow=10)

OUTPATH = Path("/data/jenna/rain-cell-stats-analysis/figures/cell_histograms_v20250704")
OUTPATH.mkdir(parents=True, exist_ok=True)

## Query data from database


In [ ]:
# Extract raincells with statistics from the database
# and pivot the data to have a wide format with quantity.statistic as columns

START_DATE = datetime(2021, 5, 1)
END_DATE = datetime(2023, 10, 1)

FETCH_QUANTITIES = [
    "vil",
    "rate",
    "dist_from_radars",
]

SQL_QUERY = """

 SELECT cell."timestamp",
    cell.identifier,
    cell.method,
    -- cell.geometry,
    ST_Area(cell.geometry) / 1e6 AS area_km2,
    stats.quantity,
    stats.statistic,
    stats.value
   FROM raincells.stormcells cell
     JOIN raincells.stormcell_rasterstats stats ON cell.identifier = stats.identifier AND cell."timestamp" = stats."timestamp" AND cell.method::text = stats.method::text

    WHERE 1 = 1
    AND cell."timestamp" >= '{start_date}'
    AND cell."timestamp" <= '{end_date}'
    AND ((stats.quantity IN ({quantities})) or (stats.quantity like 'zdrcol%') or (stats.quantity like 'et45ml%'))
    AND stats.statistic IN ('mean', 'median', 'min', 'max', 'count')

"""

with Session(ENGINE) as session:
    query = SQL_QUERY.format(
        start_date=START_DATE.strftime("%Y-%m-%d %H:%M"),
        end_date=END_DATE.strftime("%Y-%m-%d %H:%M"),
        quantities=",".join([f"'{q}'" for q in FETCH_QUANTITIES]),
    )
    DATA = pl.read_database(
        query=query,
        connection=session.bind,
    )

    DATA = DATA.with_columns(pl.format("{}.{}", "quantity", "statistic").alias("on")).pivot(
        on="on",
        # index=["type", "t0_node", "timestamp", "level", "identifier", "method", "area", "event"],
        index=set(DATA.columns) - set(["quantity", "statistic", "value"]),
        values=["value"],
    )

In [4]:
DATA

identifier,area_km2,method,timestamp,zdrcol_custom_filt_d5.min,dist_from_radars.min,rate.median,rate.mean,vil.median,vil.count,rate.count,vil.max,rate.max,vil.mean,et45ml_d1.count,et45ml_unique_d3.median,et45ml_unique_d3.count,et45ml_unique_d3.mean,et45ml_unique_d3.max,et45ml_unique_d3.min,et45ml_unique_d1.median,et45ml_unique_d1.count,et45ml_unique_d1.mean,et45ml_unique_d1.max,et45ml_unique_d1.min,et45ml_d1.mean,et45ml_d1.max,et45ml_d1.min,et45ml_d5.count,et45ml_unique_d5.median,et45ml_unique_d5.count,et45ml_unique_d5.mean,et45ml_unique_d5.max,et45ml_unique_d5.min,et45ml_d5.mean,et45ml_d5.max,et45ml_d5.min,…,et45ml.max,et45ml.mean,et45ml.count,et45ml.min,zdrcol_custom_filt_unique_d1.min,zdrcol_custom_filt_unique_d1.max,zdrcol_custom_filt_unique_d1.mean,zdrcol_custom_filt_unique_d1.count,zdrcol_custom_filt_unique_d1.median,zdrcol_custom_filt_unique_d3.min,zdrcol_custom_filt_unique_d3.max,zdrcol_custom_filt_unique_d3.mean,zdrcol_custom_filt_unique_d3.count,zdrcol_custom_filt_unique_d3.median,zdrcol_custom_filt_unique_d5.min,zdrcol_custom_filt_unique_d5.max,zdrcol_custom_filt_unique_d5.mean,zdrcol_custom_filt_unique_d5.count,zdrcol_custom_filt_unique_d5.median,zdrcol_custom_filt_d1.count,zdrcol_custom_filt_d1.min,zdrcol_custom_filt_d1.median,zdrcol_custom_filt_d1.max,zdrcol_custom_filt_d3.min,zdrcol_custom_filt_d3.max,zdrcol_custom_filt_d3.mean,zdrcol_custom_filt_d3.count,zdrcol_custom_filt_d3.median,zdrcol_custom_filt_d5.max,zdrcol_custom_filt_d5.mean,zdrcol_custom_filt_d5.count,zdrcol_custom_filt_d1.mean,zdrcol_custom_filt_d5.median,zdrcol_custom_filt.max,zdrcol_custom_filt.median,zdrcol_custom_filt.count,zdrcol_custom_filt.mean
i64,f64,str,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,10.002483,"""opencv_vil_1.0:minArea_10:clus…",2021-05-01 11:35:00,800.0,39.249542,21.242031,21.312607,1.0,17.0,18.0,1.5,31.0,1.029412,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,800.0,800.0,800.0,12.0,800.0,800.0,800.0,800.0,12.0,800.0,800.0,800.0,800.0,12.0,800.0,12.0,800.0,800.0,800.0,800.0,800.0,800.0,12.0,800.0,800.0,800.0,12.0,800.0,800.0,800.0,800.0,12.0,800.0
1,23.00571,"""opencv_vil_1.0:minArea_10:clus…",2023-05-01 00:05:00,null,235.739644,18.41,18.173825,1.0,31.0,34.0,1.5,25.0,1.016129,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,12.002979,"""opencv_vil_1.0:minArea_10:clus…",2023-05-01 00:10:00,null,237.536917,15.79,15.821498,1.0,12.0,20.0,1.0,19.870001,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,14.003476,"""opencv_vil_1.0:minArea_10:clus…",2023-05-01 04:55:00,null,99.64689,12.215,12.682917,1.0,23.0,24.0,2.0,19.08,1.282609,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,13.503352,"""opencv_vil_1.0:minArea_10:clus…",2023-05-01 05:00:00,null,101.069789,14.68,14.053201,1.5,20.0,25.0,2.5,24.959999,1.575,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,

## Filter cells that are > 140km from the nearest radar


In [5]:
DATA_CLOSE_TO_RADARS = DATA.filter(pl.col("dist_from_radars.min") < 140)

DATA_CLOSE_TO_RADARS

identifier,area_km2,method,timestamp,zdrcol_custom_filt_d5.min,dist_from_radars.min,rate.median,rate.mean,vil.median,vil.count,rate.count,vil.max,rate.max,vil.mean,et45ml_d1.count,et45ml_unique_d3.median,et45ml_unique_d3.count,et45ml_unique_d3.mean,et45ml_unique_d3.max,et45ml_unique_d3.min,et45ml_unique_d1.median,et45ml_unique_d1.count,et45ml_unique_d1.mean,et45ml_unique_d1.max,et45ml_unique_d1.min,et45ml_d1.mean,et45ml_d1.max,et45ml_d1.min,et45ml_d5.count,et45ml_unique_d5.median,et45ml_unique_d5.count,et45ml_unique_d5.mean,et45ml_unique_d5.max,et45ml_unique_d5.min,et45ml_d5.mean,et45ml_d5.max,et45ml_d5.min,…,et45ml.max,et45ml.mean,et45ml.count,et45ml.min,zdrcol_custom_filt_unique_d1.min,zdrcol_custom_filt_unique_d1.max,zdrcol_custom_filt_unique_d1.mean,zdrcol_custom_filt_unique_d1.count,zdrcol_custom_filt_unique_d1.median,zdrcol_custom_filt_unique_d3.min,zdrcol_custom_filt_unique_d3.max,zdrcol_custom_filt_unique_d3.mean,zdrcol_custom_filt_unique_d3.count,zdrcol_custom_filt_unique_d3.median,zdrcol_custom_filt_unique_d5.min,zdrcol_custom_filt_unique_d5.max,zdrcol_custom_filt_unique_d5.mean,zdrcol_custom_filt_unique_d5.count,zdrcol_custom_filt_unique_d5.median,zdrcol_custom_filt_d1.count,zdrcol_custom_filt_d1.min,zdrcol_custom_filt_d1.median,zdrcol_custom_filt_d1.max,zdrcol_custom_filt_d3.min,zdrcol_custom_filt_d3.max,zdrcol_custom_filt_d3.mean,zdrcol_custom_filt_d3.count,zdrcol_custom_filt_d3.median,zdrcol_custom_filt_d5.max,zdrcol_custom_filt_d5.mean,zdrcol_custom_filt_d5.count,zdrcol_custom_filt_d1.mean,zdrcol_custom_filt_d5.median,zdrcol_custom_filt.max,zdrcol_custom_filt.median,zdrcol_custom_filt.count,zdrcol_custom_filt.mean
i64,f64,str,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,10.002483,"""opencv_vil_1.0:minArea_10:clus…",2021-05-01 11:35:00,800.0,39.249542,21.242031,21.312607,1.0,17.0,18.0,1.5,31.0,1.029412,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,800.0,800.0,800.0,12.0,800.0,800.0,800.0,800.0,12.0,800.0,800.0,800.0,800.0,12.0,800.0,12.0,800.0,800.0,800.0,800.0,800.0,800.0,12.0,800.0,800.0,800.0,12.0,800.0,800.0,800.0,800.0,12.0,800.0
1,14.003476,"""opencv_vil_1.0:minArea_10:clus…",2023-05-01 04:55:00,null,99.64689,12.215,12.682917,1.0,23.0,24.0,2.0,19.08,1.282609,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,13.503352,"""opencv_vil_1.0:minArea_10:clus…",2023-05-01 05:00:00,null,101.069789,14.68,14.053201,1.5,20.0,25.0,2.5,24.959999,1.575,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,14.003476,"""opencv_vil_1.0:minArea_10:clus…",2023-05-01 07:20:00,null,43.765379,14.73,14.553182,1.0,21.0,22.0,1.0,16.67,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,15.003724,"""opencv_vil_1.0:minArea_10:clus…",2023-05-01 07:25:00,null,44.742316,14.635,14.542501,1.0,23.0,24.0,1.0,16.110001,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null

In [ ]:
from pprint import pprint

# zdrcol_thresholds = np.arange(800, 2000 + 1, 200)
# et45ml_thresholds = np.arange(0, 2000 + 1, 200)
zdrcol_thresholds = [
    1000,
]
et45ml_thresholds = [
    1000,
]

stats = ["median"]
zdrcol_quantities = [
    "zdrcol_custom_filt_unique_d1",
]
et45ml_quantities = [
    "et45ml_unique_d1",
]


# variables = {"zdrcol": "zdrcol_custom_filt.median", "et45ml": "et45ml.median"}

variables = {}
variables["zdrcol"] = {}
for q in zdrcol_quantities:
    for s in stats:
        variables["zdrcol"][f"{q}.{s}"] = f"{q}.{s}"

variables["et45ml"] = {}
for q in et45ml_quantities:
    for s in stats:
        variables["et45ml"][f"{q}.{s}"] = f"{q}.{s}"


conditions_conf_zdrcol = {}
for var, var_quantities in variables["zdrcol"].items():
    conditions_conf_zdrcol.update(
        {
            f"{var}:{thresh}": {
                var: {
                    "min": thresh,
                },
                "dist_from_radars.min": {"max": 140.0},
            }
            for thresh in zdrcol_thresholds
        }
    )

# Create conditions for et45ml
conditions_conf_et45ml = {}
for var, var_quantities in variables["et45ml"].items():
    conditions_conf_et45ml.update(
        {
            f"{var}:{thresh}": {
                var: {
                    "min": thresh,
                },
                "dist_from_radars.min": {"max": 140.0},
            }
            for thresh in et45ml_thresholds
        }
    )


condition_conf = {**conditions_conf_zdrcol, **conditions_conf_et45ml}
pprint(condition_conf)

{'et45ml_unique_d1.median:1000': {'dist_from_radars.min': {'max': 140.0},
                                  'et45ml_unique_d1.median': {'min': 1000}},
 'zdrcol_custom_filt_unique_d1.median:1000': {'dist_from_radars.min': {'max': 140.0},
                                              'zdrcol_custom_filt_unique_d1.median': {'min': 1000}}}


# Calculate describe statistics for each group for each cell size interval


In [10]:
cell_area_bins = np.arange(10, 101, 10)  # 10, 20, ..., 100 km2

filtering_conditions = {}

lower_p = "5%"
upper_p = "90%"
round_decimals = 2

for cond_name, cond in condition_conf.items():
    print(f"Processing condition: {cond_name}")

    # Filter trajectories based on the condition
    datafilter = graph_utils.build_filter_from_conditions(cond)
    filtered_data = DATA.filter(pl.all_horizontal(datafilter)).to_pandas()

    # Group by area bins
    area_groups = filtered_data.groupby(
        pd.cut(filtered_data["area_km2"], bins=cell_area_bins, right=False, include_lowest=True), observed=True
    )
    filtering_conditions[cond_name] = condition_conf[cond_name]
    filtering_conditions[f"{cond_name}:control"] = {}

    for area_bin, group in area_groups:

        stats = group.describe(percentiles=[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])[
            ["area_km2", "rate.median", "rate.mean", "vil.median", "vil.mean"]
        ]

        pprint(f"Statistics for condition '{cond_name}' in area bin {area_bin}:")
        pprint(stats)

        # Store the statistics in the filtering_conditions dictionary
        filtering_conditions[f"{cond_name}:control"][f"area_{area_bin.left}_{area_bin.right}"] = {
            "area": {
                "min": area_bin.left.item(),
                "max": area_bin.right.item(),
            },
            "rate.mean": {
                "min": np.round(stats["rate.mean"].loc[lower_p], round_decimals).item(),
                "max": np.round(stats["rate.mean"].loc[upper_p], round_decimals).item(),
            },
            "dist_from_radars.min": {"max": 140.0},
            f"{cond_name.split(':')[0]}": {"isnull": True},
        }

Processing condition: zdrcol_custom_filt_unique_d1.median:1000


("Statistics for condition 'zdrcol_custom_filt_unique_d1.median:1000' in area "
 'bin [10, 20):')
           area_km2   rate.median     rate.mean    vil.median      vil.mean
count  19502.000000  19502.000000  19502.000000  19502.000000  19502.000000
mean      14.401046     17.031738     19.970284      1.730976      1.985943
min       10.002483      0.720000      0.809000      1.000000      1.000000
1%        10.002483      5.085650      5.562162      1.000000      1.000000
5%        10.002483      7.428154      8.021813      1.000000      1.045455
10%       10.502607      8.830000      9.542763      1.000000      1.138889
25%       11.502855     11.553346     12.985904      1.250000      1.384615
50%       14.503600     15.562120     17.968317      1.500000      1.785714
75%       17.004220     20.775000     24.774327      2.000000      2.344828
90%       18.504593     26.857618     33.044072      2.500000      3.083333
95%       19.004717     32.128479     38.865583      3.000000     

In [11]:
print(f"Number of conditions: {len(filtering_conditions)}")

Number of conditions: 4


In [ ]:
import yaml

print(yaml.dump(filtering_conditions, default_flow_style=False))

outfile = Path(
    "/home/users/ritvanen/koodaus/rain-cell-stats-analysis/config/plots/swiss-data/filtering_conditions_v20250814.yaml"
)
with open(outfile, "w") as f:
    yaml.dump(filtering_conditions, f, default_flow_style=False)

et45ml_unique_d1.median:1000:
  dist_from_radars.min:
    max: 140.0
  et45ml_unique_d1.median:
    min: 1000
et45ml_unique_d1.median:1000:control:
  area_10_20:
    area:
      max: 20
      min: 10
    dist_from_radars.min:
      max: 140.0
    et45ml_unique_d1.median:
      isnull: true
    rate.mean:
      max: 41.96
      min: 10.26
  area_20_30:
    area:
      max: 30
      min: 20
    dist_from_radars.min:
      max: 140.0
    et45ml_unique_d1.median:
      isnull: true
    rate.mean:
      max: 41.12
      min: 10.76
  area_30_40:
    area:
      max: 40
      min: 30
    dist_from_radars.min:
      max: 140.0
    et45ml_unique_d1.median:
      isnull: true
    rate.mean:
      max: 40.62
      min: 11.31
  area_40_50:
    area:
      max: 50
      min: 40
    dist_from_radars.min:
      max: 140.0
    et45ml_unique_d1.median:
      isnull: true
    rate.mean:
      max: 41.06
      min: 12.02
  area_50_60:
    area:
      max: 60
      min: 50
    dist_from_radars.min:
      